# Test ETradeBot

In [19]:
import ast
import sys
import keyring

In [20]:
sys.path.insert(0, '../')
from strategies.strategy import Strategy
from etrade.etrade import ETrade
from execute.execute import Execute
from bot import Bot

## Authentication

In [3]:
# Retrieve credentials from keyring
consumer_key = keyring.get_password("etrade", "consumer_key")
consumer_secret = keyring.get_password("etrade", "consumer_secret")
sandbox_key = keyring.get_password("etrade", "sandbox_key")
sandbox_secret = keyring.get_password("etrade", "sandbox_secret")
web_username = keyring.get_password("etrade", "web_username")
web_password = keyring.get_password("etrade", "web_password")
account_id = keyring.get_password("etrade", "account_id")
etrade_cookie = ast.literal_eval(keyring.get_password("etrade", "cookie"))
account_id_key = keyring.get_password("etrade", "account_id_key")

In [24]:
prints = True
if prints:
    print('Consumer key: {}'.format(consumer_key))
    print('Consumer secret: {}'.format(consumer_secret))
    print('Sandbox key: {}'.format(sandbox_key))
    print('Sandbox secret: {}'.format(sandbox_secret))
    print('Web usernam: {}'.format(web_username))
    print('Web password: {}'.format(web_password))
    print('Account ID: {}'.format(account_id))
    print('ETrade cookie: {}'.format(etrade_cookie))
    print('Account ID key: {}'.format(account_id_key))

Consumer key: 0ee920d4cf28d0733965b3b0d1e0fd35
Consumer secret: e89d034b79ee59bb0cb37b5e5d8efba0003142215195ff704717e8e305f70029
Sandbox key: 0a621bc4756c47385eee930f6d95d8a4
Sandbox secret: 0dc682564f7f21d379ab01d173bad474e47e216efd5c60021f4c2ac913037696
Web usernam: betapeg
Web password: e#u.fgfE4KW6kGB
Account ID: 54134394
ETrade cookie: {'name': 'SWH', 'value': 'ETWLCUST1-be5fbc48-91fb', 'domain': '.etrade.com', 'secure': True, 'httpOnly': True}
Account ID key: rET_Y2Zn1COrVwzcy6idvA


In [4]:
# Define the strategy name
strategy_name = 'example_strategy'

In [5]:
# Configuration
dev=False  # Set to False for production, True for sandbox
headless=True  # Set to False to show browser window, True to hide
browser='chrome'  # Set to 'chrome' or 'edge'
preview=True  # Set to False to execute trades, True to preview
prints=True  # Set to True to print information to console

In [6]:
# Initialize API objects
etrade = ETrade(
    consumer_key, consumer_secret, web_username, web_password, account_id, etrade_cookie,
    sandbox_key, sandbox_secret, dev, headless, browser
)
strategy = Strategy(prints, strategy_name)
execute = Execute(etrade)

## Strategy

In [7]:
new_portfolio = strategy.run_strategy()

------------------------------------
Risk Free Rate: 3.49%

Maximum Sharpe Portfolio:
Sharpe Ratio: 0.6244

Weights:
SPY    30.00%
HYG    30.00%
TIP    19.27%
TLT    15.22%
VNQ     5.15%
Name: Maximum Sharpe Portfolio, dtype: object

Results:
Expected_Return         8.91%
Expected Volatility     8.68%
Sharpe_Ratio           0.6244
Name: Results, dtype: object

Long/Short Ratio:
Portfolio weights sum: 99.64%
Long weights sum: 99.64%
Short weights sum: 0.0%


In [18]:
new_portfolio

SPY    0.3000
HYG    0.3000
TIP    0.1927
TLT    0.1522
VNQ    0.0515
Name: Maximum Sharpe Portfolio, dtype: float64

## Current Portfolio

In [11]:
# Get current portfolio
current_portfolio = etrade.get_portfolio_data(account_id_key)[[
    'pctOfPortfolio', 'quantity', 'positionType'
]]
current_portfolio

,pctOfPortfolio,quantity,positionType
symbolDescription,,,
OIH,-0.136460,-5,SHORT
VIOV,0.532872,67,LONG
IJT,-0.151577,-14,SHORT
PSCD,0.183159,21,LONG
VIOG,0.345203,37,LONG
SPSM,0.163663,44,LONG
FYX,0.071438,9,LONG


In [13]:
# Get buying power
buying_power = etrade.get_buying_power(account_id_key, prints=prints)


Buying Power: $10,154.26


## Implement Strategy

In [14]:
# Calculate new portfolio shares
new_portfolio_shares = execute.calculate_shares(new_portfolio, buying_power, prints=prints)


Trade Instructions:
HYG    $3,022.00
SPY    $2,865.73
TIP    $1,984.50
TLT    $1,595.55
VNQ      $581.28
Name: Trade Amounts, dtype: object

Position Value: $10,049.06
Leftover: $105.20

SPY     7
HYG    40
TIP    18
TLT    15
VNQ     7
Name: Shares, dtype: object


In [16]:
# Execute trades
print('')
if preview:
    print('Previewing trades...')
else:
    print('Executing trades...')
place_order = execute.execute_trades(
    current_portfolio, new_portfolio_shares, account_id_key, preview, prints
)


Previewing trades...
BUY_TO_COVER -5 shares of OIH
SELL 67 shares of VIOV
BUY_TO_COVER -14 shares of IJT
SELL 21 shares of PSCD
SELL 37 shares of VIOG
SELL 44 shares of SPSM
SELL 9 shares of FYX
BUY 7 shares of SPY
BUY 40 shares of HYG
BUY 18 shares of TIP
BUY 15 shares of TLT
BUY 7 shares of VNQ


In [17]:
place_order

,orderAction,priceType,quantity,orderTerm,marketSession
symbol,,,,,
OIH,BUY_TO_COVER,MARKET,5,GOOD_FOR_DAY,REGULAR
VIOV,SELL,MARKET,67,GOOD_FOR_DAY,REGULAR
IJT,BUY_TO_COVER,MARKET,14,GOOD_FOR_DAY,REGULAR
PSCD,SELL,MARKET,21,GOOD_FOR_DAY,REGULAR
VIOG,SELL,MARKET,37,GOOD_FOR_DAY,REGULAR
SPSM,SELL,MARKET,44,GOOD_FOR_DAY,REGULAR
FYX,SELL,MARKET,9,GOOD_FOR_DAY,REGULAR
SPY,BUY,MARKET,7,GOOD_FOR_DAY,REGULAR
HYG,BUY,MARKET,40,GOOD_FOR_DAY,REGULAR
